
##  Open Source LLM 업무 매뉴얼 QA 시스템 구현 - 랭체인(LangChain), 올라마(ollama), 구글 젬마(gemma)

youtube: https://www.youtube.com/watch?v=GLM73CbEVaY

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_ollama import ChatOllama

model = ChatOllama(model = "gemma:7b-instruct", temperature = 0.2)

response = model.invoke("겨울철에 내한성이 강한 나무에는 어떤 것이 있을까?")

In [3]:
print(response.content)


겨울철에 내한성이 강한 나무는 다음과 같습니다.

* **동백나무**
* **오동나무**
* **거제나무**
* **느린잎버들**
* **꽃나무**
* **참나무**
* **피나크나무**


In [ ]:
! pip install rapidocr Pillow

In [6]:
import numpy as np 

from langchain_community.document_loaders import PyPDFLoader


file_path = "data\건강생활_24_1월호.pdf"
loader = PyPDFLoader(file_path)
pages = loader.load()
content = pages[0].page_content 
print(content)

2024
01
운동과 건강
근감소증 파헤치기:
예방을 위한 측정 및 운동 방법 
정신과 건강
정리가 안 되고 실수투성이인 나
성인 ADHD일까
피부와 건강
다양한 탈모 유형과 
적절한 치료 방법


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma 

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100,
)

docs = text_splitter.split_documents(pages)

embeddings = HuggingFaceEmbeddings(
    model_name = 'jhgan/ko-sroberta-nli',
    model_kwargs = {'device':'cpu'},
    encode_kwargs = {'normalize_embeddings':True},
)

vectorstore = Chroma.from_documents(docs, embeddings)

In [8]:
query = "근 감소증의 원인은 무엇이간요?"

retriever = vectorstore.as_retriever(search_kwargs={'k': 3})
docs = retriever.get_relevant_documents(query)

print(len(docs))
print(docs[0].page_content, docs[0].metadata)


3
와 영양 불충분으로 근육이 잘 합성되지 않고 점차 감소하게 되면서 발생한다.
2010년도 초까지만 해도 근감소증 진단 방법이 없었지만, 세계적으로 노령 인구가 급증하면서 
근감소증을 진단하는 기준이 생겼으며 최근에는 국내외에서 질환으로 인정하고 있다. 현재 세계 
전체 인구 중 약 10%, 우리나라의 경우는 65세 이상 노인 인구 중 약 11%가 근감소증 진단을 받았
다. 세계보건기구(WHO)는 근감소증의 위험성을 인지하여 국제 질병으로 분류하였으며, 우리나라
에서도 2016년부터 병원에서 진단을 시작했다.
근감소증은 악력·종아리 둘레·하지 근력 측정 같은 물리적 검사와 근감소증 관련 설문 검사 등
을 통해 확인할 수 있다. 한 예로, 종아리 둘레는 남성 34cm 미만, 여성은 33cm 미만인 경우 또는 
‘근감소증 자가진단 설문지’ 점수가 4점 이상인 경우에는 근감소증 가능성을 의심해 볼 수 있다. 종
아리 둘레 측정과 설문지 검사는 집에서도 쉽게 할 수 있으니 꼭 확인해 보자.
근감소증	자가진단	설문지(SARC-F)
항목 질문 점수
근력 무게 4.5kg(배 한 상자)를 들어서 나르는 것이 얼마나 어려운가요?
0점 전혀 어렵지 않다.
1점 좀 어렵다.
2점 매우 어렵다. 
또는 할 수 없다.
보행	보조 방 한쪽 끝에서 다른 쪽 끝까지 걷는 것이 얼마나 어려운가요?
의자에서	
일어나기
의자(휠체어)에서 일어나 침대(잠자리)로, 혹은 침대(잠자리)에서 
일어나 의자(휠체어)로 이동하는 것이 얼마나 어려운가요?
계단	오르기 10개의 계단을 쉬지 않고 오르는 것이 얼마나 어려운가요?
낙상 지난 1년 동안 몇 번이나 넘어졌나요?
0점 전혀 없다.
1점 1~3회 
2점 4회 이상
상지근력 측정(악력), 하지근력 측정(의자에서 5번 앉았다 일어섰다 반복) 을 통해 신체 기능을 확인
하는 방법도 있다. 악력의 경우 남성은 28kg 이하, 여성은 18kg 이하, 하지근력은 5번의 움직임을 {'creationdate': '2023-12-21T13:02:17+09:00', 

C:\Users\user2\AppData\Local\Temp\ipykernel_21656\3810167028.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


In [27]:
print(docs[1].page_content,docs[1].metadata)


2024
01
운동과 건강
근감소증 파헤치기:
예방을 위한 측정 및 운동 방법 
정신과 건강
정리가 안 되고 실수투성이인 나
성인 ADHD일까
피부와 건강
다양한 탈모 유형과 
적절한 치료 방법 {'creationdate': '2023-12-21T13:02:17+09:00', 'creator': 'Adobe InDesign CS6 (Windows)', 'moddate': '2023-12-21T15:13:16+09:00', 'page': 0, 'page_label': '1', 'producer': 'Adobe PDF Library 10.0.1', 'source': 'data\\건강생활_24_1월호.pdf', 'total_pages': 19, 'trapped': '/False'}


In [9]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [10]:
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

In [11]:
def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

In [12]:
rag_chain = ({'context': retriever | format_docs, 'question':RunnablePassthrough()}
              | prompt | model | StrOutputParser())

In [13]:
query = "근감소증의 원인은 무엇이며 치료법은?"
response = rag_chain.invoke(query)

In [16]:
print(response)

**근감소증의 원인:**
근감소증은 영양 불충분으로 근육이 잘 합성되지 않고 점차 감소하게 되면서 발생한다.

**치료법:**
근감소증을 해결하기 위한 약물 요법은 개발되지 않았기 때문에 의료의 도움을 받아 해결하기 어렵다. 따라서 현재 할 수 있는 방법은 근력 운동 및 단백질이 풍부한 건강한 음식을 먹는 것뿐이다.


In [1]:
# modules/vectorstore.py
import numpy as np
import pickle
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
import faiss

# Initialize FAISS index (change dim to match your embedding size)

#index = faiss.IndexFlatL2(dim)
#embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
ollama_embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
)

# 임베딩 차원 크기를 계산
dimension_size = len(ollama_embeddings.embed_query("hello world"))
print(dimension_size)
# Load existing index and metadata if available

# FAISS 벡터 저장소 생성
db = FAISS(
    embedding_function=ollama_embeddings,
    index=faiss.IndexFlatL2(dimension_size),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

# add documents to faiss vector store
def store_documents(documents):
    db.add_documents(documents = documents
    )
         

C:\Users\user2\AppData\Local\Temp\ipykernel_3064\3128098432.py:15: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  ollama_embeddings = OllamaEmbeddings(


768


In [9]:
from langchain_ollama import ChatOllama

model = ChatOllama(model = "gemma:7b-instruct", temperature = 0.2)

response = model.invoke("겨울철에 내한성이 강한 나무에는 어떤 것이 있을까?")

In [3]:
from transformers import pipeline, AutoModelForCausalLM

In [10]:

# Initialize the pipeline
chat_pipeline = pipeline('text-generation')

# Generate a response
user_input = "Hello, how are you?"
response = chat_pipeline(user_input, max_length=50)
print(response[0])

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'generated_text': 'Hello, how are you? What happened?" [Mocking her] "Just a little…" [The two women, both naked, slowly return home with each other.] "Oh! Oh! Yes." I looked across at you and you fell to'}


In [11]:
print(response[0]['generated_text'])

Hello, how are you? What happened?" [Mocking her] "Just a little…" [The two women, both naked, slowly return home with each other.] "Oh! Oh! Yes." I looked across at you and you fell to


In [14]:
context = """졸중 엄마가 먹는 약?\n얼마 전 후배의 급한 전화를 받았다. 후배는 “어머니가 뇌졸중으로 쓰러진 후 치료를 \n받고 입원했는데, 그동안 어떤 약을 먹었는지  알 수가 없다. 어머니 집에 갈 수도 없\n고, 가더라도 처방전이 있을지 잘 모른다.”라고 호소했다. 병원에서 환자의 투약 이력\n을 알려 달라고 요청하면서 다른 병으로 복용 중인 약이 뭔지 알아야 뇌졸중 관련 약\n을 처방하는 데 도움이 된다고 했다. 새 약을 처방할 경우 기존 약과 충돌해 치명적인 \n부작용을 초래할 수도 있다. 노인은 평균 5개 이상의 약을 먹기 때문에 갑자기 입원\n하면 약 확인에 애를 먹는다. \n   필자도 작년에 정확하게 약 이름이 기억나지 않아 당황했던 경험이 있다. 지병 때\n문에 1년에 한 번 큰 병원에 가서 약을 처방받아 온다. 그런데 그 의사가 6개월 치만 \n처방한다. 나머지 6개월 치는 가까운 동네의원에 가서 처방받으라고 한다. 회사 근처 \n의원에 처방받으러 갔다. 의사에게 약 이름을 댔더니 모니터에 2.5mL, 3mL, 플러스, \nS 등 네 가지가 떴다. 이 중 뭔지 헷갈렸다. 동네의원에서 필자의 진료 이력이나 처방 \n이력을 조회할 수 없다. \n쏠쏠한 나의 건강 기록 앱\n바로 휴대폰에서 ‘PHR’ 앱을 켰다. 보건복지부가 운영하는 ‘나의건강기록(Personal \nHealth Record)’ 앱이다. 본인인증 후 투약정보를 누르니 지난 1년 치 이력이 죽 떴다. \n약 이름을 찾았고 무사히 처방받을 수 있었다. 이 앱은 국민건강보험공단의 진료 이\n력과 건강검진 이력, 건강보험심사평가원의 투약 이력, 질병관리청의 예방접종 이력\n을 통합적으로 조회·저장하거나 활용할 수 있는 모바일 애플리케이션이다. 갤럭시폰\n이나 아이폰 둘 다 쓸 수 있다. 구글 플레이스토어(안드로이드), 앱스토어(iOS)에서 ‘나\n의건강기록’을 검색해서 설치하면 된다. \n   진료는 1년, 건 강검진은 10년, 투약 이력은 1년 치를 조회할 수 있고, 기록을 저장해 \n원하는 곳으로 전송할 수 있다. 간단한 본인인증 절차를 거치면 된다. 24시간 운영하\n는 약국, 야간에 진료하는  병원, 주변의 응급실을 찾는 기능도 탑재돼 있다. 이 글의 \n맨 앞에 언급한 후배에게도 PHR 앱을 알려 줬더니 얼마 지나지 않아 “해결했다.”라는 \n메시지를 받았다. \n해외에서, 응급실 에서 유용\n투약한 약의 이름이 영어로도 나오기 때문에 해외에 근무하거나 여행할 때 유용하게 활용할 수 있\n다. 교통사고를 당해 응급실에 실려 갈 때 더욱 유용하다. 희귀병을 앓는 환자에게 반드시 피해야 할 \n약물이 있다. 희귀병 환자는 여러 가지 약을 복용한다. 응급실에서 이런 걸 알지 못하면 기초적인 검\n사를 하기 어렵다. 실제로 한 여성 희귀병 환자가 응급실에 실려 갔고, 거기서 PHR을 구동해 대학병\n원이 처방한 희귀병 약뿐 아니라 동네의원이 처방한 두통약을 확인했다. 진료 이력도 도움이 됐다.\n   건강보험심사평가원의 ‘건강e음’ 앱도 있다. ‘내가 먹는 약! 한눈에’ 메뉴는 PHR의 투약 정보란과 \n같은 정보를 제공한다. 건강e음의 ‘내 진료정보 열람’에는 진료·투약 이력 외 총진료비 내역이 나온\n다. 건강보험이 적용된 진료의 비용이다. 총진료비 중 건강보험이 적용된 금액, 본인이 낸 금액, 병원 \n방문 일수 등을 알 수 있다.\n위암 잘 보는 병원 찾을 수 있다\n‘병원·약국 찾기’ 기능도 쓸 만하다. 네이버 같은 포털 사이트에서는 상호만 알려 준다. 건강e음에는 \n해당 의료기관의 감기 항생제 처방률, 약품 가짓수, 주사제 처방률 등의 평가 등급이 나온다. 1등급이\n면 항생제를 덜 쓰고, 처방하는 약이 많지 않고, 주사제도 덜 쓰는 곳이다. 이 앱에 들어가면 ‘병원 평\n가 정보’가 있다. 고혈압·당뇨병 같은 만성질환, 대장암·유방암·폐암·위암 등 29가지 질병 진료 평가 \n결과가 들어 있다. 가령 당뇨병을 누르고 서울 중구를 선택한 뒤 검색하면 당뇨병 진료 평가에서 양\n호 등급을 받은 의료기관 리스트가 뜬다.\n    ‘약! 찍어 보는 안심 정보’ 코너도 활 용해 봄 직하다. 약국에서 조제한 약을 보면 모양이나 색깔이 \n비슷한 게 많다. 약국에서 약의 효능을 자세히 설명해 주지 않는다. 처방전에 적힌 약 이름을 넣거나 \n약의 바코드를 찍으면 해당 약의 효능·성분·저장법·유효기간 등의 정보가 뜬다. \n '진료비 확인 요청'은 의사들이 싫어하는 코너이다. 의료기관에서 부담한 진료비에 건강보험이 적용\n되는지 확인해서 권리 구제를 받을 수 있다. 환자가 전액 부담한 진료비, 비급여 진료비 등이 제대로 \n적용됐는지를 확인한다. 건보가 적용되는데도 적용하지 않았거나, 건보와 환자에게 이중으로 부담\n시킨 경우를 가려 낸다. 그런 것으로 확인되면 진료비를 돌려받을 수 있다.\n갑자기 쓰러진 엄마\n복용 중인 약 찾으려면\n신성식  중앙일보 복지전문기자 · 논설위원\n특별 기고\n• 보건복지부, ‘나의건강기록’\n•건강보험심사평가원, ‘건강e음’"), Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.4 (Windows)', 'creationdate': '2024-06-21T15:43:47+09:00', 'moddate': '2024-06-21T15:47:26+09:00', 'trapped': '/False', 'source': 'uploaded/건강생활_24_7월호.pdf', 'total_pages': 19, 'page': 12, 'page_label': '13'}, page_content='24 25\n고용노동부의 ‘2022년도 근로자 건강진단 실시 결과’에 따르면 특수건강진단을 받은 근로자는 245만 명이며, 이 중 이\n상 소견 근로자는 132만 명이다. 즉, 특수건강진단 수검자 2명 중 1명은 건강에 적신호가 켜진 셈이다. 이상 소견 중 가장 \n많은 것은 소음성 난청이고, 유기화합물 중독, 진폐증, 금속류 중독, 산·알카리/가스상 물질 중독 순 으로 나타났다.\n산업 근로자들이 받아야 하는 건강진단\n우리나라 직장인은 1년 또는 2년마다 건강검진을 받아야 한다. 국민건강보험법(제52조)에 따른 일반건강검진과 별\n개로 사업주는 모든 근로자에게 일반건강진단(산업안전보건법 제129조)을, 그리고 소음·분진·화학물질·야간작업 등 \n건강에 유해한 업무 환경에 종사하는 근로자에게는 특수건강진단(산업안전보건법 제130조)을 주기적으로 실시하도\n록 규정하고 있기 때문이다. 산업안전보건법은 산업재해 예방과 근로자들의 안전과 보건을 위해 사업주 의무 사항\n을 규정한 법으로, 일반건강진단의 경우 비사무직은 1년에 1회, 사무직은 2년에 1회 실시해야 하며, 국민건강보험\n법에 따른 국가 일반건강검진으로 갈음할 수 있다.\n   특수건강진단은 심한 소음이나 유해화학물질, 각종 분진 등에 노출될 우려가 큰 작업장에서 일하거나 야간작업\n을 하는 사람들을 대상으로 한 건강검진이다. 이는 업무와 관련하여 발생할 수 있는 직업병을 예방하고, 조기에 발\n견하여 근로자의 건강을 보호하고 유지하기 위함이며 ‘특수건강진단/배치 전 건강진단/수시건강진단/임시건강진\n단’ 4종류로 나뉜다. 이 중 특수건강진단은 ‘산업안전보건법 시행규칙 [별표22]’에 명시된 특수건강진단 대상 유해\n인자(181종) 종류에 따라 주기적(6~24개월)으로 실시한다.\n특수건강진단\n산업 근로자에게 필수\n신동천  하나로의료재단 예방의학과 · 직업환경의학과 전문 의\n건강 정보\n특수건강진단 대상 유해인자\n1. 화학적 인자(유기화합물, 금속류, 산 및 알카리류 등) 164종\n2. 분진(곡물 분진, 광물성 분진, 면 분진, 목재 분진, 용접 흄, 유리 섬유, 석면 분진) 7종\n3. 물리적 인자(소음, 진동, 방사선, 고기압, 저기압, 유해광선 등) 8종\n4. 야간작업 2종\n   가.  6개월간 밤 12시부터 오전 5시까지의 시간을 포함하여 계속되는 8시 간 작업을 \n        월 평균 4회 이상 수행하는 경우\n   나.  6개월간 오후 10시부터 다음 날 오전 6시 사이의 시간 중 작업을 \n        월 평균 60시간 이상 수행하는 경우\n산업안전 보건법 시행규칙 [별표22]\n   그 밖에 근로자가 해당 업무에 적합한지 판단하기 위해 업무 배치 전 실시하는 건강진단과 해당 유해인자\n에 의한 건강 장해 의심 증상 또는 의학적 소견이 있는 근로자를 대상으로 실시하는 수시건강진단 그리고 동\n일 근무자와 유사한 질병이 발생한 경우, 직업병 유소견자가 다수 발생하거나 발생할 우려가 있는 경우, 지방\n고용노동관서의 장이 필요하다고 판단하는 경우 실시하는 임시건강진단이 있다.\n직업병, 조기 발견 및 예방 중요\n대표적인 직업병 중 하나인 소음성 난청은 연속적이고 강한 소음에 장기간 노출되어 발생하는 것으로 건설\n업·제조업 현장 근로자, 사격 훈련이 많은 군인·경찰관, 사이렌 소리에 지속적으로 노출되는 소방관 등 다양\n한 직업군에서 나타난다. 소음성 난청은 초기 자각 증상이 없어 질환이 악화되고 나서야 인지하기 쉬우며, \n뚜렷한 치료법이 없어 예방만이 최선이다.\n   최근 생활 양상이 변화하면서 야간에 근무하는 근로자가 증가함에 따라 이들의 건강 또한 위협받고 있다. \n야간근무는 생체리듬의 혼란을 야기하고 호르몬 등 인체 기능 조절 물질의 주기성을 변화시켜 장기간 노출 \n시 기존 질환을 악화시키거나 새롭게 질병을 일으킬 수  있으며, 고혈압 등 심혈관계 질환과 소화기 장애 그\n리고 유방암 등 일부 암 유발과 관련이 있다. 그 밖에 직업병에는 급성중독, 호흡기질환, 심뇌혈관질환, 정신\n질환 등이 있으며, 업 무상 질병자 수는 매년 증가하고 있어 각별한 주의가 필요하다.\n   직업병을 예방하기 위해서는 자신의 직업 특성을 이해하고, 검진 제도를 적극 활용하여 건강 상태를 주기\n적으로 확인해야 한다. 또한 건강검진을 받는 것에 그치지 말고 검진 결과를 꼼꼼히 확인하여 이상 소견이 \n있다면 정밀 검사를 통해 중증 질환으로 진행되지 않도록 예방하는 것이 무엇보다 중요하 다. \n사업주는 산업재해 예방을 위해 건강검진을 적극 독려해야 하며, 근로자들이 하루의 대부분을 작업장에서 \n보내는 만큼 작업 환경이 근로자에게 미치는 영향을 주의 깊게 살펴보고 안전하고 건강한 작업 환경을 조성\n하는 데 최선을 다해야 한다.\n   특수건강진단은 산업안전보건법의 규정에 따라 인력, 시설 및 장비 등의 요건을 갖춰 고용노동부의 \n지정을 받은 의료기관에서만 가능하다. 지정 기관은 산업안전보건공단 및 고용노동부 홈페이지\n에서 확인할 수 있다
"""

In [18]:
from modules.faiss_db import search_similarity

def create_prompt(query):   
    # Search query
    results = search_similarity(query, k=1)
    context = ""
    for result in results:
        context += result.page_content
    # Formulate the prompt
    
    prompt = f"Given the context: {context}\n\nQ: {query}\nA:"
    
    return prompt

In [24]:
from langchain.schema.output_parser import StrOutputParser

def query_chain_invoke(query):
    prompt = create_prompt(query)
    print(prompt) 
    model = ChatOllama(model = "gemma:7b-instruct", temperature = 0.2)
    prompt_callable = lambda _: prompt  # Create a callable for the prompt
    chain = ( prompt_callable | model | StrOutputParser())
    response = chain.invoke({})
    return response

In [25]:
query = "뇌졸증은 무엇인가요요?"

response = query_chain_invoke(query)
print(response)

Given the context: 

Q: 뇌졸증은 무엇인가요요?
A:
뇌졸증은 뇌의 일부 또는 전체 영역에 산소 공급이 원활하지 못하여 발생하는 질병입니다. 이는 뇌 기능에 장애를 초래하고, 심한 경우 사망에 이르기도 합니다.


In [ ]:
import fitz  # PyMuPDF

# Open the PDF file
pdf_document = fitz.open('path_to_your_pdf.pdf')

# Extract text from each page
for page_num in range(len(pdf_document)):
    page = pdf_document.load_page(page_num)
    text = page.get_text()
    print(f"Page {page_num + 1} Text:\n{text}\n")


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the model and tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Example text from the book
text = "Artificial intelligence is the simulation of human intelligence in machines."

# Create a prompt for question generation
input_text = f"generate question: {text}"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate questions
outputs = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)
question = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Question:", question)
